In [1]:
import datasets

/data/minpeter/github.com/minpeter/mirco-ko-llama/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset1 = datasets.load_dataset('HAERAE-HUB/KOREAN-WEBTEXT', split='train')
dataset1

Generating train split: 100%|██████████| 1284879/1284879 [00:19<00:00, 65731.50 examples/s]


Dataset({
    features: ['text', 'source', 'token_count', '__index_level_0__'],
    num_rows: 1284879
})

In [3]:
dataset2 = datasets.load_dataset('blueapple8259/c4-ko-cleaned-2', split='train')
dataset2

Generating train split: 100%|██████████| 2261464/2261464 [01:46<00:00, 21276.61 examples/s]


Dataset({
    features: ['text'],
    num_rows: 2261464
})

In [4]:
dataset3 = datasets.load_dataset('HAERAE-HUB/KOREAN-SyntheticText-1.5B', split='train')
dataset3

Generating train split: 100%|██████████| 1552370/1552370 [00:12<00:00, 127132.02 examples/s]


Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 1552370
})

In [5]:
# maywell/korean_textbooks 데이터셋에서 분류기로 3점 이상의 데이터만 수집
dataset4 = datasets.load_dataset("devngho/korean-textbooks-edu", name="scored_over_3", split="train")
dataset4

Generating train split: 100%|██████████| 1735255/1735255 [00:11<00:00, 149967.43 examples/s]


Dataset({
    features: ['text', 'score'],
    num_rows: 1735255
})

In [6]:
# 청와대 국민청원
dataset5 = datasets.load_dataset("heegyu/korean-petitions", split="train")
# dataset5에서 content 필드명을 text로 변경
dataset5 = dataset5.rename_column("content", "text")
dataset5

Generating train split: 100%|██████████| 436660/436660 [00:00<00:00, 548302.28 examples/s]


Dataset({
    features: ['category', 'begin', 'end', 'text', 'num_agree', 'petition_idx', 'status', 'title'],
    num_rows: 436660
})

In [14]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3, dataset4, dataset5])
combined_dataset

Dataset({
    features: ['text', 'source', 'token_count', '__index_level_0__', 'score', 'category', 'begin', 'end', 'num_agree', 'petition_idx', 'status', 'title'],
    num_rows: 7270628
})

In [ ]:
import os
import re
from bs4 import BeautifulSoup
import hanja
from bs4 import MarkupResemblesLocatorWarning
import warnings

# =============================================================================
# PART 1: 모든 전처리 함수 정의
# =============================================================================

def clean_text(text):
    """
    논의된 모든 텍스트 정제 및 정규화 규칙을 순서대로 적용하는 함수
    """
    # 1-1. HTML 태그 제거
    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # 1-2. URL 및 이메일 주소 제거
    text = re.sub(r'https?://\S+|www\.\S+', '[URL_PLACEHOLDER]', text)
    text = re.sub(r'\S+@\S+', '[EMAIL_PLACEHOLDER]', text)
    
    # 1-3. 한자를 한글로 변환
    text = hanja.translate(text, 'substitution')
    # 이후 단어(단어), 단어 (단어) 와 같이 단어 뒤에 바로 같은 단어가 괄호로 감싸져 오는 경우, 뒤에 (단어) 를 제거하도록 필터 추가
    text = re.sub(r'(\S+)\s+\(\1\)', r'\1', text)

    # 1-4. 불필요한 특수문자 제거 (한글, 영어, 숫자, 아스키코드 기호는 모두 허용)
    text = re.sub(r'[^ ㄱ-ㅣ가-힣a-zA-Z0-9!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]', '', text)
    
    # 1-5. 반복 문자 처리 (ㅋㅋ, ㅎㅎ 등)
    text = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1{2,}', r'\1\1', text)
    
    return text

# 중복 제거를 위한 전역 세트(set) 선언
seen_texts = set()

def is_high_quality_and_unique(example):
    """
    품질 필터링(길이)과 중복 제거를 동시에 수행하는 함수
    """
    text = example['text']
    
    # 2-1. 길이 필터링: 텍스트 길이가 100글자 미만이면 탈락
    if len(text) < 100:
        return False
    
    # 2-2. 중복 필터링: 이미 등장한 텍스트면 탈락
    if text in seen_texts:
        return False
    
    # 모든 필터를 통과한 경우, seen_texts에 추가하고 통과 처리
    seen_texts.add(text)
    return True


max_num_proc= int(os.cpu_count() / 3)


print("\n2. 텍스트 정제 및 정규화를 시작합니다... (.map)")
cleaned_dataset = combined_dataset.map(
    lambda example: {'text': clean_text(example['text'])},
    num_proc=max_num_proc,
)
print("✅ 텍스트 정제 완료")
print("정제 후 데이터셋 정보:", cleaned_dataset)

# --- 3. 품질 및 중복 필터링 (.filter) ---
# 정제된 텍스트를 기준으로 길이 필터링 및 중복 제거를 수행합니다.
print("\n3. 품질 및 중복 필터링을 시작합니다... (.filter)")
final_dataset = cleaned_dataset.filter(
    is_high_quality_and_unique,
    num_proc=1 # 'seen_texts' 세트는 전역 변수이므로 다중 처리(num_proc > 1) 시 충돌할 수 있습니다.
                # 대용량 데이터 처리 시에는 다른 중복 제거 방식이 필요할 수 있습니다.
)
print("✅ 필터링 완료")
print("\n--- 최종 결과 ---")
print("최종 데이터셋 정보:", final_dataset)


2. 텍스트 정제 및 정규화를 시작합니다... (.map)


Map (num_proc=42):   2%|▏         | 130695/7270628 [00:06<03:27, 34431.13 examples/s]/tmp/ipykernel_345646/1048091155.py:18: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  text = BeautifulSoup(text, "html.parser").get_text()
Map (num_proc=42):  10%|█         | 737374/7270628 [00:23<03:10, 34333.34 examples/s]/tmp/ipykern

✅ 텍스트 정제 완료
정제 후 데이터셋 정보: Dataset({
    features: ['text', 'source', 'token_count', '__index_level_0__', 'score', 'category', 'begin', 'end', 'num_agree', 'petition_idx', 'status', 'title'],
    num_rows: 7270628
})

3. 품질 및 중복 필터링을 시작합니다... (.filter)


Filter:  13%|█▎        | 957000/7270628 [00:29<02:29, 42251.66 examples/s]

In [ ]:
# text 컬럼만 남기고 나머지 컬럼 제거
columns_to_remove = [col for col in cleaned_dataset.column_names if col != "text"]
text_only_dataset = cleaned_dataset.remove_columns(columns_to_remove)
text_only_dataset

In [ ]:
shuffled_dataset = text_only_dataset.shuffle(seed=5768112)

In [ ]:
shuffled_dataset.push_to_hub("minpeter/pretrain-korean-dedup", split="train")